### Test inputs

In [77]:
inputs = [
    '.......S.......',
    '...............',
    '.......^.......',
    '...............',
    '......^.^......',
    '...............',
    '.....^.^.^.....',
    '...............',
    '....^.^...^....',
    '...............',
    '...^.^...^.^...',
    '...............',
    '..^...^.....^..',
    '...............',
    '.^.^.^.^.^...^.',
    '...............',
]



## Task 1

In [76]:
start_point = inputs[0].index("S")
map = {".": 0, "^": 1, "S": 0}
diagram = [[map[ch] for ch in row] for row in inputs]


splitter_nodes = [[start_point]]
path = []
counter = 0
for row in diagram:
    
    # if no splinters do not iterate
    if sum(row)>0:
        for col_id, ch in enumerate(row):
            # First splitter
            if ch == 1 and len(path) == 0 and col_id == start_point:
                path += [col_id-1, col_id+1]
                counter += 1
            # Following splitters
            elif ch == 1:
                path += [col_id-1, col_id+1]
                counter += 1
        splitter_nodes.append(sorted(list(set(path))))
        path = []
        
print(counter)
print(path)
print(splitter_nodes)


22
[]
[[7], [6, 8], [5, 7, 9], [4, 6, 8, 10], [3, 5, 7, 9, 11], [2, 4, 6, 8, 10, 12], [1, 3, 5, 7, 11, 13], [0, 2, 4, 6, 8, 10, 12, 14]]


## Task 2

In [132]:
start_point = inputs[0].index("S")
map = {".": 0, "^": 1, "S": 0}
diagram = [[map[ch] for ch in row] for row in inputs]

# Create counter for columns
counter = {start_point: 1,}

for row in diagram:
    # Find all indices of splitters
    index = {c for c in counter if row[c] == 1}

    # Loop through
    for i in index:
        # Add both left and right if not in counter
        # otherwise add from the current path
        if i-1 not in counter:
            counter[i-1] = counter[i]
        else:
            counter[i-1] += counter[i]
        
        if i+1 not in counter:
            counter[i+1] = counter[i]
        else:
            counter[i+1] += counter[i]

        # Reset current column value
        del counter[i]
        print(counter)

sum(counter.values())

{6: 1, 8: 1}
{6: 1, 7: 1, 9: 1}
{7: 2, 9: 1, 5: 1}
{7: 2, 5: 1, 8: 1, 10: 1}
{7: 2, 8: 1, 10: 1, 4: 1, 6: 1}
{8: 3, 10: 1, 4: 1, 6: 3}
{8: 3, 4: 1, 6: 3, 9: 1, 11: 1}
{8: 3, 6: 3, 9: 1, 11: 1, 3: 1, 5: 1}
{8: 3, 9: 1, 11: 1, 3: 1, 5: 4, 7: 3}
{8: 3, 9: 1, 11: 1, 5: 4, 7: 3, 2: 1, 4: 1}
{8: 4, 11: 1, 5: 4, 7: 3, 2: 1, 4: 1, 10: 1}
{8: 4, 5: 4, 7: 3, 2: 1, 4: 1, 10: 2, 12: 1}
{8: 4, 7: 3, 2: 1, 4: 5, 10: 2, 12: 1, 6: 4}
{8: 4, 7: 3, 4: 5, 10: 2, 12: 1, 6: 4, 1: 1, 3: 1}
{8: 4, 7: 3, 4: 5, 10: 2, 6: 4, 1: 1, 3: 1, 11: 1, 13: 1}
{8: 4, 7: 7, 4: 5, 10: 2, 1: 1, 3: 1, 11: 1, 13: 1, 5: 4}
{8: 4, 7: 7, 4: 5, 10: 2, 3: 1, 11: 1, 13: 1, 5: 4, 0: 1, 2: 1}
{8: 4, 7: 7, 4: 6, 10: 2, 11: 1, 13: 1, 5: 4, 0: 1, 2: 2}
{8: 4, 7: 7, 4: 10, 10: 2, 11: 1, 13: 1, 0: 1, 2: 2, 6: 4}
{8: 11, 4: 10, 10: 2, 11: 1, 13: 1, 0: 1, 2: 2, 6: 11}
{8: 11, 4: 10, 10: 2, 11: 1, 0: 1, 2: 2, 6: 11, 12: 1, 14: 1}


40

## Task 2 (Slow solution)

WAY too slow for the actual input

In [36]:
from collections import defaultdict
start_point = inputs[0].index("S")
map = {".": 0, "^": 1, "S": 0}
diagram = [[map[ch] for ch in row] for row in inputs[::2]]

# Build graph from the given input
graph = defaultdict(list)
row_paths = []
for row_id, row in enumerate(diagram):
    cur_paths = []
    for col_id, ch in enumerate(row):
        # If splitter
        if ch == 1:
            # Add left and right node to graph
            lr_nodes = [(row_id+1, col_id-1), (row_id+1, col_id+1)]
            graph[(row_id, col_id)] = lr_nodes
            
            # current paths
            cur_paths += lr_nodes

            # Mark any previous path split by removing from the list
            if (row_id, col_id) in row_paths:
                row_paths.remove((row_id, col_id))

    # If any unsplit paths, add those to graph as direct lines
    if len(row_paths) > 0:
        for val in row_paths:
            graph[(val[0], val[1])] = [(val[0]+1, val[1])]
            cur_paths += [(val[0]+1, val[1])]

    # Save paths visited this iteration to row_paths
    row_paths = list(set(cur_paths))
    cur_paths = []
    

### Depth first search

In [37]:
# graph
class Graph:
    def __init__(self, graph):
        self.graph = graph
        self.path_counter = 0
            
    def DFS(self, node, visited):
        visited = visited.copy()
        visited += [node]

        # No children
        if node not in self.graph:
            self.path_counter += 1
            # print(visited)
            return

        for child in self.graph[node]:
            if child not in visited:
                self.DFS(child, visited)



graphh = Graph(graph)
visited = []
graphh.DFS((1,start_point), visited)

graphh.path_counter

40